In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import random
from torch.optim.lr_scheduler import CosineAnnealingLR

from data import *
from encoder_decoder import *
from train import *

/home/student/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SEED = 31989101
HIDDEN_SIZE = 256
MAX_INGR_LEN = 150
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## ensuring reproducibility
def reset_rng():
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

reset_rng()

print(f"Using device: {DEVICE}")

Using device: cuda


In [4]:
# to easily read ingredients and instructions
pd.set_option('display.max_colwidth', 2000)

In [5]:
data_root = "./Cooking_Dataset"

train_df_orig = pd.read_csv(os.path.join(data_root, "train.csv"), usecols=['Ingredients', 'Recipe'])
dev_df_orig = pd.read_csv(os.path.join(data_root, "dev.csv"), usecols=['Ingredients', 'Recipe'])
test_df_orig = pd.read_csv(os.path.join(data_root, "test.csv"), usecols=['Ingredients', 'Recipe'])

In [6]:
train_df = preprocess_data(train_df_orig, max_ingr_len=MAX_INGR_LEN)

Number of data samples before preprocessing: 101340
Number of data samples after preprocessing: 99036 (97.726%)


In [7]:
vocab = Vocabulary()
vocab.populate(train_df)
vocab.n_unique_words

100%|██████████| 99036/99036 [00:08<00:00, 12164.60it/s]


44683

In [8]:
recipe_ds = RecipeDataset(train_df, vocab)

In [9]:
encoder = EncoderRNN(vocab.n_unique_words, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index[PAD_WORD]).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
decoder = DecoderRNN(hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1).to(DEVICE)

In [10]:
initial_lr=0.8
min_lr = 0.01
n_epochs = 20
batch_size=128
encoder_optimizer = optim.SGD(encoder.parameters(), lr=initial_lr)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=initial_lr)
enc_scheduler = CosineAnnealingLR(encoder_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_scheduler = CosineAnnealingLR(decoder_optimizer, T_max=n_epochs, eta_min=min_lr)
epoch_losses = train(encoder, decoder, encoder_optimizer, decoder_optimizer, recipe_ds, 
                     n_epochs=n_epochs, vocab=vocab, batch_size=batch_size, 
                     enc_lr_scheduler=enc_scheduler, dec_lr_scheduler=dec_scheduler, 
                     verbose_iter_interval=10)

Starting epoch 1/20, enc lr scheduler: [0.5], dec lr scheduler: [0.5]
(Epoch 0, iter 10/1548) Average loss so far: 10.664
(Epoch 0, iter 20/1548) Average loss so far: 10.558
(Epoch 0, iter 30/1548) Average loss so far: 10.454
(Epoch 0, iter 40/1548) Average loss so far: 10.340
(Epoch 0, iter 50/1548) Average loss so far: 10.211
(Epoch 0, iter 60/1548) Average loss so far: 10.065
(Epoch 0, iter 70/1548) Average loss so far: 9.889
(Epoch 0, iter 80/1548) Average loss so far: 9.662


KeyboardInterrupt: 